# Agent RAG 示例

In [29]:
! pip install -qU langchain-openai langchain langchain_community langchainhub
! pip install chromadb==0.5.3
# 设置 OPENAI API KEY
# # export OPENAI_API_KEY=sk-...


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [30]:
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
#from langchain_chroma import Chroma
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.prompts import PromptTemplate
from string import Template
import uuid


In [31]:
# 读取 ./data/data.md 文件作为运维知识库
file_path = os.path.join('data', 'data.md')
with open(file_path, 'r', encoding='utf-8') as file:
    docs_string = file.read()

# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = text_splitter.split_text(docs_string)
print("Length of splits: " + str(len(splits)))
print(splits)

Length of splits: 11
[Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '1. 系统监控指标'}, page_content='- **交易量**: 监控每秒交易数，确保系统承载能力。\n- **响应时间**: 监控交易的平均响应时间，确保服务性能。\n- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.1 交易失败'}, page_content='- **问题描述**: 用户发起支付后，交易未能成功完成。\n- **可能原因**:\n- 网络延迟或中断\n- 支付网关服务异常\n- **解决办法**:\n- 检查网络连接，确保支付服务的网络畅通。\n- 检查支付网关服务状态，重启服务或联系服务提供商。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.2 响应时间过长'}, page_content='- **问题描述**: 用户支付请求处理时间超过正常范围。\n- **可能原因**:\n- 系统资源不足\n- 数据库查询效率低下\n- 外部服务响应慢\n- **解决办法**:\n- 增加系统资源，如CPU、内存。\n- 优化数据库查询，使用索引，减少复杂查询。\n- 与外部服务提供商沟通，优化接口性能。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.3 系统宕机'}, page_content='- **问题描述**: 支付系统完全无法访问或服务中断。\n- **可能原因**:\n- 主机硬件故障\n- 系统软件崩溃\n- 网络设备故障\n- **解决办法**:\n- 快速切换到备用服务器。\n- 检查系统日志，定位问题原因。\n- 联系硬件供应商，进行故障排查和修复。'), Document(metadata={'Head

In [32]:
# 将运维知识库的每一块文本向量化（Embedding）
# 保存到随机目录里
random_directory = "./" + str(uuid.uuid4())
embedding = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key="sk-T6SlqfUnyFytejvA3c1584F87d6343878232185e26243b1d",openai_api_base="https://api.apiyi.com/v1")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=random_directory)
vectorstore.persist()

In [45]:
# 传统 RAG 流程
retriever = vectorstore.as_retriever()
# 提示语模板
template = """使用以下上下文来回答最后的问题。
如果你不知道答案，就说不知道，不要试图编造答案。
最多使用三句话，并尽可能简洁地回答。
在答案的最后一定要说“谢谢询问！”。

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    print("匹配到的运维知识库片段：\n", "\n\n".join(doc.page_content for doc in docs))
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-4o")
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


# 传统 RAG 无法回答的问题
res = rag_chain.invoke("谁管的系统最多？")
print("\n\nLLM 回答：", res)

匹配到的运维知识库片段：
 - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。

- **代码优化**: 定期审查代码，优化算法和逻辑。
- **资源扩展**: 根据业务增长，适时扩展系统资源。
- **负载均衡**: 使用负载均衡技术，分散请求压力。

- **发现问题**: 监控系统发现异常指标。
- **快速响应**: 立即通知运维团队，进行初步诊断。
- **问题定位**: 通过日志分析，定位问题原因。
- **解决方案**: 根据问题类型，执行相应的解决方案。
- **问题解决**: 确认问题解决，恢复正常服务。
- **后续跟进**: 记录问题处理过程，进行事后分析和总结。


LLM 回答： 我不知道。谢谢提问！


In [51]:
# Agent RAG 流程

# 文本相似性检索
def search_docs(query, k=1):
    results = vectorstore.similarity_search_with_score(
        query,
        k=k,
    )
    return "\n\n".join(doc.page_content for doc, score in results)

user_query = "谁管的系统最多？"

check_can_answer_system_prompt = """
根据上下文识别是否能够回答问题，如果不能，则返回 JSON 字符串 "{"can_answer": false}"，如果可以则返回 "{"can_answer": true}"。
上下文：\n $context
问题：$question

"""
llm = ChatOpenAI(model="gpt-4o")
k = 1
docs = ""
while True:
    # 通过检索找到相关文档，每次循环增加一个检索文档数量，最大 15 个文档块
    print("第", k, "次检索")
    if k > 15:
        break
    docs = search_docs(user_query, k)
    print("匹配到的文档块: ", docs)
    template = Template(check_can_answer_system_prompt)
    filled_prompt = template.substitute(question=user_query, context=docs)
    # 检查上下文是否足够回答问题
    messages = [
        (
            "system",
            filled_prompt,
        ),
        ("human", "开始检查上下文是否足够回答问题。"),
    ]
    llm_message = llm.invoke(messages)
    content = llm_message.content
    print("\nLLM Res: ", content, "\n")
    if content == '{"can_answer": true}':
        break
    else:
        k += 1

print("匹配到能够回答问题的知识库，开始进行回答\n")

# 最终推理
final_system_prompt = """
您是问答任务的助手，使用检索到的上下文来回答用户提出的问题。如果你不知道答案，就说不知道。最多使用三句话并保持答案简洁。
"""

final_messages = [
    (
        "system",
        final_system_prompt,
    ),
    ("human", "上下文：\n"+ docs +"\n问题：" + user_query),
]
llm_message = llm.invoke(final_messages)
content = llm_message.content
print("\nLLM Final Res: ", content, "\n")

第 1 次检索
匹配到的文档块:  - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

LLM Res:  {"can_answer": false} 

第 2 次检索
匹配到的文档块:  - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。

LLM Res:  {"can_answer": false} 

第 3 次检索
匹配到的文档块:  - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。

- **代码优化**: 定期审查代码，优化算法和逻辑。
- **资源扩展**: 根据业务增长，适时扩展系统资源。
- **负载均衡**: 使用负载均衡技术，分散请求压力。

LLM Res:  {"can_answer": false} 

第 4 次检索
匹配到的文档块:  - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。

- **代码优化**: 定期审查代码，优化算法和逻辑。
- **资源扩展**: 根据业务增长，适时扩展系统资源。
